# Web scraping av polisens händelser

En vanlig organisation som ingår i journalisternas nyhetsnät är polisen. På polisens hemsida finns en [lista över händelser i Sverige](https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/). Vi kan förvisso leta i denna händelselista manuellt efter något som intresserar oss, men varför inte göra det automatiskt med hjälp av Python?

Vi kan lösa det på två sätt:

1. **Skrapa RSS-flödet**. Skrapa polisens [RSS-flöde med händelser](https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?feed=rss). Detta är lättast eftersom [RSS](https://sv.wikipedia.org/wiki/RSS) är en standard för att utbyta information på webbplatser. Då kan vi också använda koden till alla webbplatser som har RSS-flöde. Men polisens RSS-flöde är begränsat, och allt som finns på hemsidan finns inte med i RSS-flödet.
2. **Skrapa webbsidan**. Vi skrapar webbsidans HTML-kod. Då kan vi få ned allt vi vill ha. Men om polisen ändrar sin hemsida så kan vårt program slutera fungera.

Vilken ska vi välja? Vi testar båda!

## Importera bibliotek

Importera de bibliotek som behövs för att skrapa hemsidor (`BeautifulSoup4` och `lxml`).

In [3]:
# Installera bibliotek. Om det inte fungerar, testa att skriva pip3 i stället.
!pip install BeautifulSoup4
!pip install lxml

In [4]:
# Importera bibliotek.
import urllib.request
from lxml import html
from bs4 import BeautifulSoup 

# Funktion för stt skrapa HTML från webbsida.
def scrapewebpage(url):
	# Öppna URL och hämta HTML.
	web = urllib.request.urlopen(url)

	# Kontrollera att inga fel uppstod när URL öppnades.
	if (web.getcode() == 200):
		html = web.read()
		return(html)
	else:
		print("Error %s reading %s" % str(web.getcode()), url)

# Hjälpfunktion som skrapar webbsidor och gör en soppa av allt ihop.
def makesoup(url):
	html = scrapewebpage(url)
	return(BeautifulSoup(html, "lxml-xml"))

## 1. Skrapa polisens händelser från RSS-flödet

RSS-flöde: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?feed=rss

In [5]:
# Läs in RSS-flödet och spara det i variablen rss_events.
rss_events = makesoup("https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?feed=rss")

# Ta ut alla <item> från RSS-flödet.
rss_items = rss_events.find_all("item")

In [6]:
# Hur många händelser hittade vi?
print("Hittade " + str(len(rss_items)) + " händelser.")

Hittade 200 händelser.


In [7]:
# Visa senaste 10 händelserna. Plocka bort [0:10] för att visa alla.
for item in rss_items[0:10]:
    title = item.find("title").get_text()
    print(title)

2017-11-09 08:01, Trafikhinder, Umeå
2017-11-09 07:53, Trafikolycka, Ljungby
2017-11-09 07:48, Trafikolycka, Malmö
2017-11-09 07:29, Trafikolycka, vilt, Umeå
2017-11-09 07:18, Trafikolycka, vilt, Överkalix
2017-11-09 07:04, Trafikolycka, Solna
2017-11-09 07:00, Sammanfattning natt, Jämtland
2017-11-09 06:59, Rån, Västerås
2017-11-09 06:55, Vapenlagen, Motala
2017-11-09 06:54, Trafikolycka, Lomma


In [8]:
# Gå igenom varje händelse, rad för rad, och bearbeta texten.
for item in rss_items:
    title = item.find("title").get_text()                # Titel på händelsen
    link = item.find("link").get_text()                  # Länk till händelsen på polisens hemsida
    pubdate = item.find("pubdate").get_text()            # Datum händelsen publicerades
    description = item.find("description").get_text()    # Beskrivning av händelsen
    
    # Titlarna är skrivna i formatet "2017-11-07 20:40, Trafikolycka, Göteborg".
    # Då kan vi dela upp titeln i tre delar genom komma-tecknet
    # och på så vis plocka ut datum, händelse och ort.
    title_parts = title.split(",")                         # Dela sträng vid varje komma
    title_date = title_parts[0].strip()                    # Datum
    title_event = title_parts[1].strip()                   # Händelse
    title_city = title_parts[len(title_parts) - 1].strip() # Stad
    
    # Vad ska vi plocka ut? Bestäm med hjälp av if-sats.
    if title_city in ["Göteborg", "Borås"]:
        print(title_date + " " + title_city + ", " + title_event + ": " + description)

2017-11-08 20:30 Göteborg, Brand: Lägenhetsbrand i Gårdsten
2017-11-08 14:07 Göteborg, Trafikolycka: På Backavägen vid Leråkersmotet har två bilar kolliderat.
2017-11-08 11:52 Borås, Brand: Lägenhetsbrand på Trandaredsgatan.
2017-11-08 10:33 Göteborg, Rån övrigt: I Lövgärdets centrum har två personer tilltvingat sig en blå lastbil från Postnord.
2017-11-08 07:58 Göteborg, Trafikolycka: På Tunnlandsgatan har en fotgängare blivit påkörd av en bilist.
2017-11-08 06:52 Göteborg, Trafikolycka: På Lillhagsvägen har en man kört omkull med sin moped.
2017-11-08 06:40 Göteborg, Trafikolycka: På Rävebergsvägen har två bilar kolliderat.
2017-11-07 20:40 Göteborg, Trafikolycka: I korsningen Skånegatan-Ullevigatan har två bilar kolliderat
2017-11-07 15:56 Borås, Rattfylleri: Misstänkt rattfylleri på riksväg 40.
2017-11-07 14:27 Göteborg, Trafikolycka: På Marieholmsleden är tre bilar inblandade i en olycka. 
2017-11-07 13:35 Göteborg, Brand: Brand i sprängmattor på Sisjövägen.


## 2. Skrapa polisens händelser från webbsidans HTML-kod

Webbsida: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/ (notera att det finns händelser på flera sidor!)

In [9]:
# Läs in webbsidan och spara det i variablen html_events.
html_events = makesoup("https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/")

# Hur många sidor med händelser finns det?
html_pages = html_events.find_all("", "pageingnumbers")
pages = html_pages[0].get_text().strip().split(" ")
print("Hittade " + str(len(pages)) + " sidor med händelser.")

Hittade 6 sidor med händelser.


In [10]:
# Importera time för att kunna fördröja hämtningen.
import time

# Skapa en tom lista som vi ska spara händelserna i.
combined_events = []

# Gå igenom alla webbsidor.
for i in range(1, len(pages) + 1):
    # Visa meddelande om vad som händer och hur många sidor som har skrapats.
    print("Skrapar sida " + str(i) + ": https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=" + str(i))
    
    # Hämta HTML-kod från aktuell webbsida.
    page_soup = makesoup("https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=" + str(i))
    
    # Gå igenom alla händelser på sidan.
    for event in page_soup.find("ul", "page-list").find_all("li"):
        
        # Skapa ett dictionary-objekt av aktuell händelse.
        eventdict = { "title": event.find("h3").get_text().strip(),
                      "county": event.find("p").find("span", "news-date").get_text().strip(),
                      "description": event.find("p").get_text().strip(),
                      "link": "https://polisen.se" + event.find("h3").find("a").get("href")
                    }

        # Lägg till händelse i listan.
        combined_events.append(eventdict)
        
    # Lägg till 1.5 sekunders fördröjning för
    # för att inte överbelasta polisens server.
    time.sleep(1.5)

print("Klart.")

Skrapar sida 1: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=1
Skrapar sida 2: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=2
Skrapar sida 3: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=3
Skrapar sida 4: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=4
Skrapar sida 5: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=5
Skrapar sida 6: https://polisen.se/Aktuellt/Handelser/Handelser-i-hela-landet/?p=6
Klart.


In [11]:
# Hur många händelser hittade vi totalt?
print("Hittade " + str(len(combined_events)) + " händelser.")

Hittade 1110 händelser.


In [12]:
# Visa de 10 senaste händelserna. Plocka bort [0:10] för att visa alla.
for event in combined_events[0:10]:
    print(event["title"])

2017-11-09 08:01, Trafikhinder, Umeå
2017-11-09 07:53, Trafikolycka, Ljungby
2017-11-09 07:48, Trafikolycka, Malmö
2017-11-09 07:29, Trafikolycka, vilt, Umeå
2017-11-09 07:18, Trafikolycka, vilt, Överkalix
2017-11-09 07:04, Trafikolycka, Solna
2017-11-09 07:00, Sammanfattning natt, Jämtland
2017-11-09 06:59, Rån, Västerås
2017-11-09 06:58, Stöld/inbrott, Kalix
2017-11-09 06:55, Vapenlagen, Motala


In [13]:
# Gå igenom varje händelse, rad för rad, och bearbeta texten.
for event in combined_events:
    description = event["description"]   # Beskrivning av händelsen
    link = event["link"]                 # Länk till händelsen
    county = event["county"]             # Län
    
    # Titlarna är skrivna i formatet "2017-11-07 20:40, Trafikolycka, Göteborg".
    # Då kan vi dela upp titeln i tre delar genom komma-tecknet.
    title_parts = event["title"].split(" ")   # Dela händelsen efter komma-tecken
    date = title_parts[0].strip()             # Datum
    event = title_parts[2].strip()            # Händelse
    
    # Eftersom det ibland finns fler än tre komma-tecken kan vi behöva
    # hantera fel som uppstår genom try ... except.
    try:
        city = title_parts[len(title_parts) - 1].strip()  # Stad
    except:
        city = ""
         
    # Vad ska vi plocka ut? Bestäm med hjälp av if-sats.
    if city in ["Göteborg", "Stockholm"] and event in ["Rån", "Inbrott"]:
        print(date + " " + city + ", " + event)
        print(link)
        print()

2017-11-08 Göteborg, Rån
https://polisen.se/Aktuellt/Handelser/Vastra-Gotaland/2017-11-08-1033-Ran-ovrigt-Goteborg/

2017-11-07 Stockholm, Rån
https://polisen.se/Aktuellt/Handelser/Stockholms-lan/2017-11-07-1909-Ran-ovrigt-Stockholm/

2017-11-06 Stockholm, Rån
https://polisen.se/Aktuellt/Handelser/Stockholms-lan/2017-11-06-1947-Ran-ovrigt-Stockholm/

